In [1]:
from kaggle_environments import evaluate, make, utils, agent
import numpy as np
import time
from Minimax import Minimax
from Node import Node
from bitboard_utils import *

env = make("connectx", debug=True)
env.render()

Loading environment lux_ai_2022 failed: No module named 'pettingzoo'


In [2]:
def my_agent(obs, config):
    bitboard_occupied, bitboard_black = list_to_bitboard(obs.board)
    
    horizontal_masks, vertical_masks, positive_diagonal_masks, negative_diagonal_masks = get_line_masks(config.rows, config.columns, config.inarow)
    all_masks = np.hstack((horizontal_masks, vertical_masks, positive_diagonal_masks, negative_diagonal_masks))
    
    root = Node(bitboard_occupied, bitboard_black, obs.mark, config.rows, config.columns, config.inarow, obs.mark, all_masks)
    minimax = Minimax(root, 50000, silent=False)
    
    next_node = minimax.make_move()
    difference = next_node.bitboard_occupied - bitboard_occupied
    for row in range(config.rows):  # maybe it can be done better
        for col in range(config.columns):
            if difference & (1 << (col + row * config.columns)):
                return col
            
    print(f'{difference:042b}')
    print(bitboard_to_array(next_node.bitboard_occupied, next_node.bitboard_black, config.rows, config.columns))
    assert False
    return 0

In [3]:
%load_ext line_profiler

In [7]:
bitboard_occupied, bitboard_black = list_to_bitboard([0] * 42)
    
horizontal_masks, vertical_masks, positive_diagonal_masks, negative_diagonal_masks = get_line_masks(6, 7, 4)
all_masks = np.hstack((horizontal_masks, vertical_masks, positive_diagonal_masks, negative_diagonal_masks))

root = Node(bitboard_occupied, bitboard_black, 1, 6, 7, 4, 1, all_masks)
minimax = Minimax(root, 50000, silent=False)

In [8]:
%prun minimax.make_move()

time 0.7288539409637451,	eval -24.0,	visited 14090 nodes,	depth 5
next move:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]]
 

         851827 function calls (837696 primitive calls) in 0.729 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    14090    0.309    0.000    0.398    0.000 bitboard_utils.py:65(count_lines)
    14090    0.198    0.000    0.670    0.000 Node.py:47(get_children)
   251069    0.040    0.000    0.040    0.000 {built-in method builtins.bin}
   251069    0.039    0.000    0.039    0.000 {method 'count' of 'str' objects}
    98628    0.038    0.000    0.038    0.000 Node.py:6(__init__)
  14090/7    0.031    0.000    0.729    0.104 Minimax.py:37(minimax)
    10509    0.021    0.000    0.021    0.000 Node.py:24(evaluate)
    14090    0.014    0.000    0.426    0.000 Node.py:69(is_terminal)
    14090    0.010    0.000    0.408    0.000 Node.py:20(evaluate_lines)
    28181    0.010    0.000    0.010    0.000 {built-in method numpy.zeros}
    98628    0.008    0.000    0.008    0.000 {method 'append' of 'list' objects}
    22778    

In [6]:
env.reset()
result = env.run([lambda obs, config : my_agent(obs, config), my_agent])
env.render(mode="ipython", width=500, height=450)

time 0.5568151473999023,	eval -24.0,	visited 14090 nodes,	depth 5
next move:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]]
time 0.6148874759674072,	eval -24.0,	visited 15823 nodes,	depth 5
next move:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 1. 0.]]
time 0.41641879081726074,	eval -30.0,	visited 10700 nodes,	depth 5
next move:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 2. 0. 0. 1. 0.]]
time 0.6200909614562988,	eval -31.0,	visited 15244 nodes,	depth 5
next move:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0.]
 [0. 1. 2. 0. 0. 1. 0.]]
time 0.4621584415435791,	eval -36.0,	visited 11633 nodes,	depth 5
next move:
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0.

# Debug/Train your Agent

In [8]:
parameters = np.array(initial_parameters).copy()

for i in range(1000):
    noise = (np.random.rand(7) - 0.5) / 2
    
    result = env.run([
        lambda obs, config : my_agent(obs, config, parameters=parameters+noise),
        lambda obs, config : my_agent(obs, config, parameters=parameters)
    ])
    reward_white = result[-1][0].reward
    
    result = env.run([
        lambda obs, config : my_agent(obs, config, parameters=parameters),
        lambda obs, config : my_agent(obs, config, parameters=parameters+noise)
    ])
    reward_black = - result[-1][0].reward
    
    if reward_white + reward_black > 0:
        parameters += noise
    if reward_white + reward_black < 0:
        parameters -= noise
        
    print(f'rewards: {reward_white} for white, \t{reward_black} for black, \tnew parameters: {parameters}')

rewards: 0 for white, 	-1 for black, 	new parameters: [1.7834312  3.94146728 0.2867147  1.04991032 1.99295926 1.47922353
 1.13128296]
rewards: -1 for white, 	1 for black, 	new parameters: [1.7834312  3.94146728 0.2867147  1.04991032 1.99295926 1.47922353
 1.13128296]
rewards: 1 for white, 	0 for black, 	new parameters: [1.89958818 3.80820451 0.04611929 0.88028234 1.90215121 1.62281161
 0.93082036]
rewards: 1 for white, 	-1 for black, 	new parameters: [1.89958818 3.80820451 0.04611929 0.88028234 1.90215121 1.62281161
 0.93082036]
rewards: 1 for white, 	-1 for black, 	new parameters: [1.89958818 3.80820451 0.04611929 0.88028234 1.90215121 1.62281161
 0.93082036]
rewards: 1 for white, 	1 for black, 	new parameters: [ 1.84023757e+00  3.78017575e+00 -1.25175818e-04  8.68205480e-01
  2.08381815e+00  1.40147662e+00  8.24764651e-01]
rewards: 1 for white, 	1 for black, 	new parameters: [ 1.80030498  3.83747202 -0.08670207  0.8507766   1.95427877  1.18982405
  0.69777208]
rewards: 0 for white, 	

In [20]:
new_parameters = [1.69256408, 6.42524593, 1.68890947, -4.21051354, 4.01389685, 3.68415865, -0.43183429]
new_parameters = [2., 10., 0.5, 1., 5., 4., 1.]
new_parameters = [0.82016218, 2.66486931, 2.93518357, -1.39310179, 2.58320125, 3.00020255, 3.23011953]
new_parameters = [2., 5., 2.93518357, 0., 2.58320125, 3.00020255, 3.23011953]    

In [21]:
rewards_white = evaluate("connectx", [
    lambda obs, config : my_agent(obs, config, parameters=new_parameters),
    lambda obs, config : my_agent(obs, config, parameters=initial_parameters)
    ], num_episodes=10)

In [22]:
print(rewards_white)
print(np.mean(np.array(rewards_white), axis=0))

[[1, -1], [1, -1], [1, -1], [1, -1], [0, 0], [-1, 1], [1, -1], [1, -1], [1, -1], [-1, 1]]
[ 0.5 -0.5]


In [23]:
rewards_black = evaluate("connectx", [
    lambda obs, config : my_agent(obs, config, parameters=initial_parameters),
    lambda obs, config : my_agent(obs, config, parameters=new_parameters)
    ], num_episodes=10)

In [24]:
print(rewards_black)
print(- np.mean(np.array(rewards_black), axis=0))

[[1, -1], [1, -1], [0, 0], [1, -1], [1, -1], [0, 0], [0, 0], [-1, 1], [1, -1], [1, -1]]
[-0.5  0.5]


# Evaluate your Agent

In [13]:
def mean_reward(rewards):
    return sum(r[0] for r in rewards) / float(len(rewards))

# Run multiple episodes to estimate its performance.
# print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [my_agent, "random"], num_episodes=10)))
print("My Agent vs My Agent:", mean_reward(evaluate("connectx", [my_agent, my_agent], num_episodes=10)))

My Agent vs Negamax Agent: -0.4


# Write Submission File



In [7]:
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(my_agent, "submission.py")

<function my_agent at 0x7f27dafed700> written to submission.py


# Validate Submission
Play your submission against itself.  This is the first episode the competition will run to weed out erroneous agents.

Why validate? This roughly verifies that your submission is fully encapsulated and can be run remotely.

In [4]:
# Note: Stdout replacement is a temporary workaround.
import sys
out = sys.stdout
submission = utils.read_file("submission.py")
my_agent = agent.get_last_callable(submission, path=submission)
sys.stdout = out

env = make("connectx", debug=True)
env.run([my_agent, my_agent])
env.render(mode="ipython", width=500, height=450)
print("Success!" if env.state[0].status == env.state[1].status == "DONE" else "Failed...")

Success!
